In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling
from transformers import AutoConfig, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import pipeline
import time
from tqdm import tqdm
import pandas as pd
import pandas as pd
from datasets import Dataset


In [3]:
untrained_model_path = "/home/uddeshya.singh/Un-trained_Model/distilroberta-base/"
retrained_lm_model_save_dir = '/home/uddeshya.singh/Trained_model/save/'
retrained_lm_model_out_dir = '/home/uddeshya.singh/Trained_model/out/'

In [87]:
#train_address_data_path = '/home/uddeshya.singh/Roberta-MLM-data/pan_india_only_text_train.csv'


In [38]:
train_address_data_path = "/home/uddeshya.singh/10.116.1.27:9000/KERALA_with_grid_ids_train.csv"

In [39]:
df = pd.read_csv(train_address_data_path)
df = df[['cleaned_address']]
df = df.dropna(subset=['cleaned_address'])


In [56]:
print(df['cleaned_address'][20004])

kallalathil house ulladathu vayal thalikkavu kannur 1 


In [48]:
address_data = Dataset.from_pandas(df)

print(address_data)

Dataset({
    features: ['cleaned_address', '__index_level_0__'],
    num_rows: 9604394
})


In [49]:
# from datasets import load_dataset

# train_address_data_path = '/home/uddeshya.singh/Roberta-MLM-data/pan_india_only_text_train.csv'

# address_data = load_dataset("csv", data_files=train_address_data_path, split='train[:100%]')

# print(f'Loaded subset size: {len(address_data)}')

# print(address_data[:5])


In [50]:
address_data

Dataset({
    features: ['cleaned_address', '__index_level_0__'],
    num_rows: 9604394
})

In [51]:
def get_training_corpus():
    batch_size = 1000
    for i in tqdm(range(0, len(address_data), batch_size), desc="Preparing training corpus"):
        yield address_data[i: i + batch_size]["cleaned_address"]


In [52]:
# train_address_data_path = '/home/uddeshya.singh/Roberta-MLM-data/pan_india_only_text_train.csv'
# print('starting now ', time.strftime('%X %x %Z'))
# address_data = load_dataset("csv", data_files=train_address_data_path)
# print('starting now ', time.strftime('%X %x %Z'))



In [53]:
address_data

Dataset({
    features: ['cleaned_address', '__index_level_0__'],
    num_rows: 9604394
})

In [54]:
# def get_training_corpus():
#     batch_size = 1000
#     for i in tqdm(range(0, len(address_data["train"]), batch_size), desc="Preparing training corpus"):
#         yield address_data["train"][i: i + batch_size]["cleaned_address"]


In [55]:
tokeniser_save = "/home/uddeshya.singh/Trained_model/Tokeniser/"
print('Starting tokenizer training: ', time.strftime('%X %x %Z'))
vocab_size = 50265
training_corpus = get_training_corpus()

pretrained_tokenizer = AutoTokenizer.from_pretrained(untrained_model_path)
tokenizer = pretrained_tokenizer.train_new_from_iterator(training_corpus, vocab_size)
tokenizer.save_pretrained(tokeniser_save)
print('Tokenizer trained: ', time.strftime('%X %x %Z'))

Starting tokenizer training:  12:28:03 08/05/24 IST


Preparing training corpus: 100%|██████████| 9605/9605 [00:43<00:00, 223.08it/s]


Tokenizer trained:  12:29:56 08/05/24 IST


In [58]:
tokenizer = AutoTokenizer.from_pretrained(tokeniser_save)


In [59]:
txt = "kallalathil house ulladathu vayal thalikkavu kannur"
tokens = tokenizer(txt)['input_ids']

converted = tokenizer.convert_ids_to_tokens(tokens)
print(converted)

['<s>', 'kall', 'alathil', 'Ġhouse', 'Ġull', 'adathu', 'Ġvayal', 'Ġthalikkavu', 'Ġkannur', '</s>']


In [60]:
len(tokenizer)

50265

In [61]:
pan = "/home/uddeshya.singh/PAN-India-MLM/pan_india_train_threshold_save_dir"
tokenizer_pan = AutoTokenizer.from_pretrained(pan)

tokens = tokenizer_pan(txt)['input_ids']

converted = tokenizer_pan.convert_ids_to_tokens(tokens)
print(converted)

['<s>', 'kall', 'al', 'athil', 'Ġhouse', 'Ġull', 'ad', 'athu', 'Ġvayal', 'Ġthali', 'kkavu', 'Ġkannur', '</s>']


In [62]:
tokenizer_untrained = AutoTokenizer.from_pretrained(untrained_model_path)


In [63]:

tokens = tokenizer_untrained(txt)['input_ids']

converted = tokenizer_untrained.convert_ids_to_tokens(tokens)
print(converted)

['<s>', 'k', 'all', 'al', 'ath', 'il', 'Ġhouse', 'Ġu', 'll', 'ad', 'ath', 'u', 'Ġv', 'ay', 'al', 'Ġth', 'al', 'ikk', 'av', 'u', 'Ġk', 'ann', 'ur', '</s>']


In [29]:
MAX_LEN = 100
def tokenize(sentence):
    return tokenizer(
        sentence["cleaned_address"],
        truncation=True,
        max_length=MAX_LEN
    )

# Apply tokenization with progress bar
tokenized_datasets = address_data.map(
    lambda x: tokenize(x),
    batched=True,
    remove_columns=address_data.column_names,
    desc="Tokenizing dataset"
)
print('Dataset tokenized: ', time.strftime('%X %x %Z'))

Tokenizing dataset:   0%|          | 0/6914 [00:00<?, ?ba/s]

KeyboardInterrupt: 

In [56]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# Verify the data collator
out = data_collator([tokenized_datasets[i] for i in range(32)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([32, 20])
attention_mask shape: torch.Size([32, 20])
labels shape: torch.Size([32, 20])


In [57]:
config = AutoConfig.from_pretrained(untrained_model_path, vocab_size=len(tokenizer))
model = RobertaForMaskedLM(config=config)

loading configuration file /home/uddeshya.singh/Un-trained_Model/distilroberta-base/config.json
Model config RobertaConfig {
  "_name_or_path": "/home/uddeshya.singh/Un-trained_Model/distilroberta-base/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



In [58]:
NO_OF_EPOCHS = 10
training_args = TrainingArguments(
    output_dir=retrained_lm_model_out_dir,
    overwrite_output_dir=True,
    num_train_epochs=NO_OF_EPOCHS,
    per_device_train_batch_size=64,
    logging_steps=10000,
    save_steps=25000,
    save_total_limit=5,
    # evaluation_strategy="steps",
    # load_best_model_at_end=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [59]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets
    # eval_dataset=tokenized_datasets, # Commented out since it's the same as train
)

In [60]:
print('Starting training: ', time.strftime('%X %x %Z'))
start_time = time.time()
trainer.train()
end_time = time.time()
print('Training complete: ', time.strftime('%X %x %Z'))
print(f'Total training time: {end_time - start_time} seconds')

***** Running training *****
  Num examples = 16797907
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 656170


Starting training:  17:49:44 07/29/24 IST


KeyboardInterrupt: 